# Stop Loss Component Testing
This notebook tests the stop loss functionality of the bull put spread strategy in isolation.

In [ ]:
# Import necessary libraries
from AlgorithmImports import *
from datetime import datetime, timedelta
import pandas as pd
import json

# Initialize the research environment
qb = QuantBook()
spy = qb.AddEquity("SPY")
option = qb.AddOption("SPY")

# Set up the option filter
option.SetFilter(-10, 10, 0, 5)  # Wider range for testing

# Set the simulation date
test_date = datetime(2023, 10, 2)
qb.SetStartDate(test_date - timedelta(days=5))  # Small lookback
qb.SetEndDate(test_date + timedelta(days=5))
qb.SetCash(100000)

In [ ]:
# Define the stop loss check function (copied from main.py)
def check_stop_loss(qb, current_debit, initial_credit, stop_loss_multiplier):
    """
    Checks if the current position has hit the stop loss threshold.
    
    Args:
        qb: The QuantBook instance for logging
        current_debit: Current cost to close the spread
        initial_credit: Initial credit received
        stop_loss_multiplier: Multiplier for stop loss (e.g., 2.0 for 2x credit)
        
    Returns:
        bool: True if stop loss is triggered, False otherwise
    """
    if current_debit is None or initial_credit <= 0:
        qb.Debug("Invalid inputs for stop loss check")
        return False
        
    current_loss = current_debit * 100  # Convert to dollars
    stop_loss_threshold = initial_credit * stop_loss_multiplier
    
    qb.Debug(f"Stop Loss Check - Current Loss: ${current_loss:.2f}, Threshold: ${stop_loss_threshold:.2f}")
    
    return current_loss >= stop_loss_threshold

In [ ]:
# Test the stop loss with sample data
test_cases = [
    (0.50, 1.00, 2.0, False),  # $50 loss, $100 credit, 2x multiplier = $200 threshold
    (2.50, 1.00, 2.0, True),   # $250 loss, $100 credit, 2x multiplier = $200 threshold
    (1.50, 0.75, 2.0, True),   # $150 loss, $75 credit, 2x multiplier = $150 threshold
    (0.49, 0.25, 2.0, False)   # $49 loss, $25 credit, 2x multiplier = $50 threshold
]

results = []
for current_debit, initial_credit, multiplier, expected in test_cases:
    triggered = check_stop_loss(qb, current_debit, initial_credit, multiplier)
    results.append({
        'current_debit': current_debit,
        'initial_credit': initial_credit,
        'multiplier': multiplier,
        'expected': expected,
        'actual': triggered,
        'passed': triggered == expected
    })

# Display results
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Get historical option data for more realistic testing
def get_option_chain(qb, symbol, expiry):
    """Get option chain for a specific expiry"""
    qb.Debug(f"Getting option chain for {symbol} expiring {expiry.date()}")
    
    # Set the filter for the specific expiry
    option = qb.AddOption(symbol, resolution=Resolution.Minute)
    option.SetFilter(lambda u: u.strikes(-5, 5).expiration(0, 1))
    
    # Get the option chain
    chain = qb.CurrentSlice.OptionChains.get(option.Symbol, None)
    if chain is None:
        qb.Debug("No option chain data available")
        return None
        
    return chain

# Test with real option data
def test_with_real_data(qb, test_date):
    # Get option chain for the test date
    chain = get_option_chain(qb, "SPY", test_date)
    if chain is None:
        qb.Debug("Failed to get option chain")
        return
        
    # Find a put spread (simplified example)
    puts = [contract for contract in chain if contract.Right == OptionRight.Put]
    if len(puts) < 2:
        qb.Debug("Not enough put options found")
        return
        
    # Sort by strike price
    puts.sort(key=lambda x: x.Strike)
    
    # Select a spread (simplified - in practice, you'd use delta-based selection)
    short_put = puts[-2]  # Second highest strike
    long_put = puts[-4]    # Fourth highest strike (2 strikes below short)
    
    initial_credit = (short_put.AskPrice - long_put.BidPrice) * 100  # Credit received
    qb.Debug(f"Initial credit: ${initial_credit:.2f}")
    
    # Simulate price movement (in a real test, you'd use historical data)
    for price_move in [0, -0.01, -0.02, -0.05, -0.10]:
        # In a real test, you'd get updated option prices here
        # This is a simplified example
        current_debit = (short_put.BidPrice * (1 + price_move) - long_put.AskPrice * (1 + price_move * 0.5))
        triggered = check_stop_loss(qb, current_debit, initial_credit, 2.0)
        qb.Debug(f"Price move: {price_move*100:.1f}% - Current debit: ${current_debit*100:.2f} - Stop loss triggered: {triggered}")

# Run the test with real data
test_with_real_data(qb, test_date)

## Next Steps

1. Run this notebook in QuantConnect Research
2. Review the debug output to verify stop loss behavior
3. Adjust the stop loss multiplier in `main.py` if needed
4. Test with different market conditions by changing the test date

## Notes
- This is a simplified test environment
- In live trading, additional factors like liquidity and slippage will affect results
- Consider testing with different stop loss multipliers to find the optimal setting